In [1]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

ROOT = Path('/home/sabina.jangirova/Documents/ML703_project/data')

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date,strict=False))
        return df

    def handle_dates(df):
        for col in df.columns: 
                if col.endswith("D"):
                    # Calculate the difference in days between each date column and date_decision
                    df = df.with_columns(
                        (pl.col("date_decision") - pl.col(col)).dt.total_days().alias(col)
                    )
                    df = df.with_columns(pl.col(col).fill_null(np.nan)) 
        # Drop date_decision column
        df = df.drop("date_decision")
#         print(df.dtypes) # for Debugging
        return df

    def filter_cols(df,base_df = None,test=False):
        #for test data
            for col in df.columns:
                if col not in ["target", "case_id", "WEEK_NUM"]:
                    isnull = df[col].is_null().mean()
                    if isnull > 0.7:
                        df = df.drop(col)
            columns_to_drop = []
            for col in df.columns:
                if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                    freq = df[col].n_unique()
                    if (freq == 1) or (freq > 100):
                        columns_to_drop.append(col)

            df = df.drop(columns_to_drop)
            return df


class Aggregator:
    
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)
        return exprs

def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.set_table_dtypes)
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                try:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
                except:
                    continue
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

In [4]:
%%time
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
    ]
}

CPU times: user 3min 39s, sys: 1min 43s, total: 5min 23s
Wall time: 44.6 s


In [5]:
df = feature_eng(**data_store) # import train data 
print("train data shape:\t", df.shape)
# gc.collect()
# spamming gc.collect praying for memory to not full
gc.collect()
df = df.pipe(Pipeline.filter_cols) # fillter column
gc.collect()
df, cat_cols = to_pandas(df) # tranform to pandas dataframe, easier to work with
gc.collect()
df = reduce_mem_usage(df) # as the name said
gc.collect()
print("train data shape:\t", df.shape)
nums=df.select_dtypes(exclude='category').columns
# IDK what is going on for now
from itertools import combinations, permutations
#df=df[nums]
nans_df = df[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
            #print(str(gg)+'-'+str(n),', ',end='')
        use.append(vx)
        #print()
    print('Use these',use)
    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    correlation_matrix = matrix.corr()
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    return groups

uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(df[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+list(df.select_dtypes(include='category').columns)
print(len(uses))
df=df[uses]

train data shape:	 (1526659, 488)
Memory usage of dataframe is 3313.73 MB
Memory usage after optimization is: 1097.80 MB
Decreased by 66.9%
train data shape:	 (1526659, 344)
Use these ['case_id', 'WEEK_NUM', 'target', 'month_decision', 'weekday_decision', 'credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'isbidproduct_1095L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L', 'max_mainoccupationinc_384A', 'max_birth_259D

## Feature engineering

In [6]:
# df.info()

In [7]:
# df.describe()

In [8]:
# missing_data = df.isnull().sum()
# print("\nMissing Data:")
# print(missing_data[missing_data > 0])

In [9]:
# print("\nCorrelation Matrix:")
# plt.figure(figsize=(15, 12))
# sns.heatmap(df.corr(), fmt=".2f", cmap='coolwarm')
# plt.show()

In [10]:
import networkx as nx
import seaborn as sns
import dowhy.gcm as gcm
from castle.algorithms import PC
from castle.algorithms.pc.pc import find_skeleton
from castle.common import GraphDAG
from castle.metrics import MetricsDAG

2024-05-11 12:03:07,167 - /home/sabina.jangirova/.conda/envs/ai701/lib/python3.8/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2024-05-11 12:03:08,579 - /home/sabina.jangirova/.conda/envs/ai701/lib/python3.8/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [11]:
def plot_graph(input_graph, node_lookup):
    '''
    Function to visualise graphs.

    Args:
        input_graph (array): Adjacency matrix representing graph
        node_lookup (dict): Dictionary containing node names.
    '''
    
    graph = nx.DiGraph(input_graph)

    plt.figure(figsize=(8,8))
    nx.draw(
    G=graph,
    node_color=COLORS[0],
    node_size=8000,
    arrowsize=17,
    with_labels=True,
    labels=node_lookup,
    font_color='white',
    font_size=9,
    pos=nx.circular_layout(graph)
    )

In [12]:
y = df['target']
df = df.drop(columns=["case_id"])

In [13]:
df[cat_cols] = df[cat_cols].astype(str)
import polars as pl
from sklearn.preprocessing import OrdinalEncoder

# Fit Ordinal Encoder on Training Data
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
encoder.fit(df[cat_cols])

# Transform Training Data
df[cat_cols] = encoder.transform(df[cat_cols])
df[cat_cols] = df[cat_cols].fillna(-1)
df[cat_cols] = df[cat_cols].astype(int)

In [14]:
df = df.fillna(0)

In [15]:
np.random.seed(42)
node_lookup = {i: column for i, column in enumerate(df.columns)}
total_nodes = len(node_lookup)

In [16]:
df = df.drop(columns=['target'])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [18]:
weeks = X_train['WEEK_NUM']
X_train = X_train.drop(columns=['WEEK_NUM'])
X_test = X_test.drop(columns=['WEEK_NUM'])

In [19]:
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

In [20]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 10,  
    "learning_rate": 0.05,
    "n_estimators": 2000,  
    "colsample_bytree": 0.8,
    "colsample_bynode": 0.8,
    "random_state": 42,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':64,
    "device_type": "cpu", 
    "verbose": 1,
}

In [21]:
import time 
import resource

def using():
    usage=resource.getrusage(resource.RUSAGE_SELF)
    return usage[2]/1024.0

In [22]:
fitted_models = []
cv_scores = []
lgbm_times = []
lgbm_memory = []
for idx_train, idx_valid in cv.split(X_train, y_train, groups=weeks):
    X_train_l, y_train_l = X_train.iloc[idx_train], y_train.iloc[idx_train]
    X_valid, y_valid = X_train.iloc[idx_valid], y_train.iloc[idx_valid]
    
    start_time = time.time()
    start_memory = using()
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train_l, y_train_l,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)] )
    lgbm_memory.append(start_memory-using())
    lgbm_times.append(start_time-time.time())
    fitted_models.append(model)
    
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores.append(auc_score)
    
print("CV AUC scores: ", cv_scores)
print("Maximum CV AUC score: ", max(cv_scores))

[LightGBM] [Info] Number of positive: 30178, number of negative: 947050
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.141069 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 37530
[LightGBM] [Info] Number of data points in the train set: 977228, number of used features: 297
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.030881 -> initscore=-3.446239
[LightGBM] [Info] Start training from score -3.446239
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.842283
[400]	valid_0's auc: 0.848083
[600]	valid_0's auc: 0.850086
[800]	valid_0's auc: 0.850895
[1000]	valid_0's auc: 0.851333
[1200]	valid_0's auc: 0.851478
Early stopping, best iteration is:
[1123]	valid_0's auc: 0.851489
[LightGBM] [Info] Number of positive: 30985, number of negative: 946298
[LightGBM] [Info] Auto-choosing row-wise multi-threading

In [23]:
from joblib import dump, load
dump(fitted_models, "./saved_models/lgbm/fitted_models.joblib")
dump(cv_scores, "./saved_models/lgbm/cv_scores.joblib")
dump(lgbm_times, "./saved_models/lgbm/lgbm_times.joblib")
dump(lgbm_memory, "./saved_models/lgbm/lgbm_memory.joblib")

['./saved_models/lgbm/lgbm_memory.joblib']

In [28]:
lgbm_times

In [24]:
# from sklearn.metrics import confusion_matrix, classification_report
# from sklearn.model_selection import StratifiedGroupKFold
# import numpy as np

# conf_matrix = None
# class_report = None

# # Predict labels for validation data
# y_pred_labels = fitted_models[3].predict(X_test)

# # Compute confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_labels)

# # Compute classification report
# class_report = classification_report(y_test, y_pred_labels)

# # Print confusion matrix and classification report
# print("Confusion Matrix:")
# print(conf_matrix)
# print("\nClassification Report:")
# print(class_report)

In [25]:
# lgb.plot_importance(fitted_models[3], importance_type="split", figsize=(10,50))
# plt.show()

In [26]:
# features = X_test.columns
# importances = fitted_models[3].feature_importances_
# feature_importance = pd.DataFrame({'importance':importances,'features':features}).sort_values('importance', ascending=False).reset_index(drop=True)

In [27]:
# for f in feature_importance.iterrows():
#     print(f)